In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import logging
import pickle
import time
import os

# Set up logging for clearer output
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- GLOBAL CONSTANTS (Must match the training environment) ---
LOOK_BACK = 45
# CRITICAL FIX: The source data must be larger than LOOK_BACK to account for dropped rows
LOOK_BACK_SOURCE = LOOK_BACK + 7

SAFETY_STOCK = 75
MIN_ORDER_QTY = 50

NUMERICAL_FEATURE_COLS = [
    'Lag_Sales_D-1', 'Lag_Sales_D-2', 'Lag_Sales_D-7', 'Lag_Inventory_D-1', 'Rolling_Mean_7D',
    'Price', 'Discount', 'Holiday/Promotion', 'Competitor Pricing',
    'Region__North', 'Region__South', 'Region__West', 'Weather Condition__Rainy',
    'Weather Condition__Snowy', 'Weather Condition__Sunny', 'Seasonality__Spring',
    'Seasonality__Summer', 'Seasonality__Winter'
]
CATEGORY_TO_ID = {'Groceries': 3, 'Toys': 4, 'Electronics': 1, 'Clothing': 0, 'Furniture': 2, 'Tools': 5, 'Books': 6, 'Cosmetics': 7}
N_NUM_FEATURES = len(NUMERICAL_FEATURE_COLS)

# --- ASSET LOADING AND INITIALIZATION ---

def load_scaler(filepath):
    """Loads a fitted MinMaxScaler object from a pickle file."""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Scaler file not found at: {filepath}. Please upload it.")
    with open(filepath, 'rb') as f:
        scaler = pickle.load(f)
    return scaler

# --- PLACEHOLDER FOR ACTUAL ASSETS ---
# NOTE: You MUST update these file paths in your Colab notebook!
MODEL_PATH = 'final_inventory_forecast_model.h5'
SCALER_X_PATH = 'scaler_x.pkl'
SCALER_Y_PATH = 'scaler_y.pkl'

try:
    # 1. Load the Keras Model
    model = tf.keras.models.load_model(MODEL_PATH)
    logging.info(f"Successfully loaded Keras model from {MODEL_PATH}.")

    # 2. Load the Scalers
    scaler_X = load_scaler(SCALER_X_PATH)
    scaler_y = load_scaler(SCALER_Y_PATH)
    logging.info("Successfully loaded MinMaxScaler objects.")

except Exception as e:
    logging.error(f"FATAL ERROR: Failed to load model or scalers. Cannot proceed with real testing.")
    logging.error(f"Error: {e}")
    raise SystemExit("Required assets not found. Please ensure model and scaler files are uploaded to Colab.")


# --- CORE PREDICTION AND RESTOCKING FUNCTION ---

def get_restock_recommendation(df_live_window, current_inventory):
    """
    Executes the full feature engineering, prediction, and restocking policy using the loaded model.
    """
    df_live_window = df_live_window.copy()

    try:
        # 1. Feature Engineering
        category = df_live_window['Category'].iloc[-1]

        # Apply Lags and Rolling Mean
        df_live_window['Lag_Sales_D-1'] = df_live_window['Units Sold'].shift(1)
        df_live_window['Lag_Sales_D-2'] = df_live_window['Units Sold'].shift(2)
        df_live_window['Lag_Sales_D-7'] = df_live_window['Units Sold'].shift(7)
        df_live_window['Lag_Inventory_D-1'] = df_live_window['Inventory Level'].shift(1)
        df_live_window['Rolling_Mean_7D'] = df_live_window['Units Sold'].shift(1).rolling(window=7).mean()

        # Drop the NaN rows resulting from the shifting
        # This will now drop 7 rows, leaving (52 - 7) = 45 rows, which is correct.
        df_live_window.dropna(subset=['Lag_Sales_D-7', 'Rolling_Mean_7D'], inplace=True)

        # Check size after drop - MUST be 45 for the model to accept it
        if len(df_live_window) != LOOK_BACK:
             raise ValueError(f"Feature engineering resulted in sequence length {len(df_live_window)}, but model expects {LOOK_BACK}.")

        # One-Hot Encoding
        categorical_features = ['Region', 'Weather Condition', 'Seasonality']
        df_ohe = pd.get_dummies(df_live_window, columns=categorical_features, drop_first=True)

        # Create Category ID input
        X_seq_cat_raw = CATEGORY_TO_ID[category]

        # Align Numerical Feature Vector
        missing_cols = set(NUMERICAL_FEATURE_COLS) - set(df_ohe.columns)
        for c in missing_cols:
            df_ohe[c] = 0

        # Extract the full sequence (should be 45 steps)
        X_seq_num_final = df_ohe[NUMERICAL_FEATURE_COLS].values

        # Scale Numerical Input
        X_seq_num_scaled = scaler_X.transform(X_seq_num_final)

        # Reshape for LSTM input: (1, 45, N_FEATURES)
        X_num_input = X_seq_num_scaled[np.newaxis, :, :]
        X_cat_input = np.array([X_seq_cat_raw])

        # 2. Model Prediction (The real deal!)
        predicted_sales_scaled = model.predict({'num_input': X_num_input, 'cat_input': X_cat_input})
        predicted_sales = scaler_y.inverse_transform(predicted_sales_scaled)[0][0]

        # 3. Restocking Policy Engine
        target_stock = predicted_sales + SAFETY_STOCK
        raw_order_quantity = target_stock - current_inventory

        if raw_order_quantity <= MIN_ORDER_QTY:
            order_quantity = 0
            status = "Sufficient Stock / Low Demand"
        else:
            # Round up to the nearest multiple of MIN_ORDER_QTY
            order_quantity = np.ceil(raw_order_quantity / MIN_ORDER_QTY) * MIN_ORDER_QTY
            status = "Order Recommended"

        return {
            "predicted_demand": round(predicted_sales, 2),
            "target_stock": round(target_stock, 2),
            "order_quantity": int(order_quantity),
            "status": status
        }

    except Exception as e:
        logging.error(f"Prediction failed in get_restock_recommendation: {e}")
        return {"error": str(e)}

# --- TEST DATA GENERATION (Requires your original retail_store_inventory.csv) ---

def create_real_historical_data(product_id, store_id, df_source):
    """Extracts the last LOOK_BACK_SOURCE rows from the real data to simulate a live feed."""
    df_filtered = df_source[
        (df_source['Product ID'] == product_id) & (df_source['Store ID'] == store_id)
    ].tail(LOOK_BACK_SOURCE)

    if len(df_filtered) < LOOK_BACK_SOURCE:
        # Changed the error check to the new, larger source size
        raise ValueError(f"Not enough historical data for {product_id}. Need {LOOK_BACK_SOURCE} rows, found {len(df_filtered)}.")

    current_inventory = df_filtered['Inventory Level'].iloc[-1]

    return df_filtered, current_inventory

# --- MAIN EXECUTION ---

# Load the source data (assuming 'retail_store_inventory.csv' is available in Colab)
try:
    df_source = pd.read_csv('retail_store_inventory.csv')
    df_source['Date'] = pd.to_datetime(df_source['Date'])
    df_source.sort_values(['Product ID', 'Date'], inplace=True)
except FileNotFoundError:
    logging.error("FATAL ERROR: 'retail_store_inventory.csv' not found. Please upload the original data file.")
    raise SystemExit("Missing original data file.")


TEST_CASES = [
    # TEST 1: P0001 (Groceries/High Demand Category) - Check if it recommends ordering
    {"product_id": "P0001", "store_id": "S001", "custom_inventory": 50, "category": "Groceries"},

    # TEST 2: P0005 (Electronics/Medium Demand Category) - Check for minimal/no order
    {"product_id": "P0005", "store_id": "S001", "custom_inventory": 300, "category": "Electronics"},

    # TEST 3: P0010 (Furniture/Low Demand Category) - Check if it recommends NO order
    {"product_id": "P0010", "store_id": "S001", "custom_inventory": 500, "category": "Furniture"},
]

print("\n" + "="*80)
print("COLAB END-TO-END PRODUCTION TEST: USING TRAINED MODEL AND SCALERS")
print("="*80)

for i, test in enumerate(TEST_CASES):
    try:
        # 1. Generate Real Historical Data Slice
        df_history, real_inventory = create_real_historical_data(test['product_id'], test['store_id'], df_source)

        # Use custom inventory level for testing the restocking policy thresholds
        inventory_to_test = test['custom_inventory']

        # 2. Run Prediction Pipeline
        result = get_restock_recommendation(df_history, inventory_to_test)

        # 3. Print Results
        predicted_demand = result.get('predicted_demand', 0)
        target_stock = result.get('target_stock', 0)
        order_quantity = result.get('order_quantity', 0)
        status = result.get('status', result.get('error', 'N/A'))

        print(f"\n--- TEST {i+1}: {test['product_id']} ({test['category']}) ---")
        print(f"Inventory Used in Test: {inventory_to_test} units")
        print(f"Predicted Demand (7-Day): {predicted_demand:.2f} units")
        print(f"Target Stock (Demand + Safety Stock): {target_stock:.2f} units (Safety Stock: {SAFETY_STOCK})")
        print(f"RESTOCK ORDER: {order_quantity} units")
        print(f"Status: {status}")

    except Exception as e:
        print(f"\n--- TEST FAILED for {test['product_id']} ---")
        print(f"Error: {e}")

print("\n" + "="*80)
print("PRODUCTION TESTING COMPLETE.")
print("="*80)



COLAB END-TO-END PRODUCTION TEST: USING TRAINED MODEL AND SCALERS
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

--- TEST 1: P0001 (Groceries) ---
Inventory Used in Test: 50 units
Predicted Demand (7-Day): 974.38 units
Target Stock (Demand + Safety Stock): 1049.38 units (Safety Stock: 75)
RESTOCK ORDER: 1000 units
Status: Order Recommended
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

--- TEST 2: P0005 (Electronics) ---
Inventory Used in Test: 300 units
Predicted Demand (7-Day): 974.04 units
Target Stock (Demand + Safety Stock): 1049.04 units (Safety Stock: 75)
RESTOCK ORDER: 750 units
Status: Order Recommended
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

--- TEST 3: P0010 (Furniture) ---
Inventory Used in Test: 500 units
Predicted Demand (7-Day): 967.11 units
Target Stock (Demand + Safety Stock): 1042.11 units (Safety Stock: 75)
RESTOCK ORDER: 550 units
Status: Order Recommended

PRODUCTION TESTING COMPLETE.
